# Match segments


1. translate one segment so that the ends are over the first one
2. compute some sort of distance


## Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.types import EDGE_ENDS_TO_CORNER
from snap_fit.config.types import EdgePos
from snap_fit.image.process import find_contours
from snap_fit.image.process import find_corners
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.image.utils import draw_contour
from snap_fit.image.utils import draw_corners
from snap_fit.image.utils import show_image_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager


## Load new data


In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = sheet_aruco.load_sheet

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

# Verify Sheets
sheets = manager.get_sheets_ls()
print(f"Managed Sheets: {len(sheets)}")
for sheet in sheets:
    print(f" - {sheet.img_fp.name}: {len(sheet.pieces)} pieces")

# Verify Pieces
pieces = manager.get_pieces_ls()
print(f"Total Pieces: {len(pieces)}")


## Get pieces and segments


In [ ]:
p1_index = 1
p2_index = 3

s1_type = EdgePos.LEFT
s2_type = EdgePos.BOTTOM

p1 = pieces[p1_index]
p2 = pieces[p2_index]
rprint(f"using {p1.name} and {p2.name}")

seg1 = p1.contour.segments[s1_type]
seg2 = p2.contour.segments[s2_type]

# 02_PXL_20251207_204130904 has index 1; left is clear edge
# 01_PXL_20251207_204113818 has index 3; should match on bottom

### Visualize the segments


In [ ]:
img_contour_seg = p1.img_bw.copy() // 10
for _ei, edge_name in enumerate(EdgePos):
    segment = p1.contour.segments[edge_name]
    points = segment.points
    color = 250 if edge_name == s1_type else 120
    draw_contour(img_contour_seg, points, color=color)
show_image_mpl(img_contour_seg, figsize=(5, 5))

img_contour_seg = p2.img_bw.copy() // 10
for _ei, edge_name in enumerate(EdgePos):
    segment = p2.contour.segments[edge_name]
    points = segment.points
    color = 250 if edge_name == s2_type else 120
    draw_contour(img_contour_seg, points, color=color)
show_image_mpl(img_contour_seg, figsize=(5, 5))

In [ ]:
# create a segment matcher so that s1 points get transformed
seg_match = SegmentMatcher(seg1, seg2)

sim = seg_match.compute_similarity()
rprint(f"got {sim=:.2f}")

# draw the internal segments
p2_img = p2.img_bw.copy() // 10
draw_contour(p2_img, seg_match.s2.points, color=120)
draw_contour(p2_img, seg_match.s1_points_transformed, color=120)
show_image_mpl(p2_img, figsize=(5, 5))